# Employee Exploration

## Setup

In [1]:
# Import Dependencies

#Import Pandas and Matplotlib
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Import SQLAlchemy Automap, ORM, and other dependencies here
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

In [3]:
# Configure Database from PostgreSQL
from config import uri

In [4]:
# Create Engine
engine = create_engine(uri)

## Reflect and Explore Database

In [5]:
# Use the Inspector to explore the database and print the table names
inspector = inspect(engine)
inspector.get_table_names()

['Employees',
 'Department_Employee',
 'Departments',
 'Department_Manager',
 'Salaries',
 'Titles']

In [6]:
# Get column names for Employees
columns = inspector.get_columns('Employees')
for c in columns:
    print(c['name'], c['type'])

emp_no INTEGER
birth_date DATE
first_name VARCHAR(30)
last_name VARCHAR(30)
gender VARCHAR
hire_date DATE


In [7]:
# Get column names for Titles
columns = inspector.get_columns('Titles')
for c in columns:
    print(c['name'], c['type'])

id INTEGER
emp_no INTEGER
title VARCHAR(60)
from_date DATE
to_date DATE


In [8]:
# Get column names for Salaries
columns = inspector.get_columns('Salaries')
for c in columns:
    print(c['name'], c['type'])

id INTEGER
emp_no INTEGER
salary INTEGER
from_date DATE
to_date DATE


In [9]:
# Reflect Database into ORM class
Base = automap_base()
Base.prepare(engine, reflect = True)

In [10]:
# Print all of the classes mapped to the Base
Base.classes.keys()

['Employees',
 'Department_Employee',
 'Departments',
 'Department_Manager',
 'Salaries',
 'Titles']

In [11]:
# for e, s in session.query(Employees).filter(Employees.emp_no == Salaries.emp_no).limit(10).all():
#     print("last_name: {} Salary: {}".format(e.last_name, s.salary))

In [12]:
# Assign the "Departments" class to a variable called Departments
Departments = Base.classes.Departments

# Assign the "Employees" class to a variable called Employees
Employees = Base.classes.Employees

# Assign the "Salaries" class to a variable called Employees
Salaries = Base.classes.Salaries

# Assign the "Titles" class to a variable called Employees
Titles = Base.classes.Titles

In [13]:
# list(engine.execute('SELECT * FROM "Employees" as e JOIN "Titles" as t ON e.emp_no = t.emp_no JOIN "Salaries" as s ON e.emp_no = s.emp_no'))

## Reflect Database and Query

In [14]:
# Reflect Database into ORM class
# Base = automap_base()
# Base.prepare(engine, reflect = True)

# Set variable to hold the database
# EmployeeSQL = Base.classes.keys()
# EmployeeSQL

In [15]:
# Start a session to query the database
session = Session(bind=engine)

In [16]:
# Query the EmployeeSQL database for Employee Name
query = session.query(Employees)
employee_df = pd.read_sql_query(query.statement, session.get_bind())

In [17]:
employee_df.head(20)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
5,10006,1953-04-20,Anneke,Preusig,F,1989-06-02
6,10007,1957-05-23,Tzvetan,Zielinski,F,1989-02-10
7,10008,1958-02-19,Saniya,Kalloufi,M,1994-09-15
8,10009,1952-04-19,Sumant,Peac,F,1985-02-18
9,10010,1963-06-01,Duangkaew,Piveteau,F,1989-08-24


In [21]:
# Query the EmployeeSQL database for Titles
query_title = session.query(Titles)
titles_df = pd.read_sql_query(query_title.statement, session.get_bind())
titles_df.head()

,id,emp_no,title,from_date,to_date
0,1,10001,Senior Engineer,1986-06-26,9999-01-01
1,2,10002,Staff,1996-08-03,9999-01-01
2,3,10003,Senior Engineer,1995-12-03,9999-01-01
3,4,10004,Engineer,1986-12-01,1995-12-01
4,5,10004,Senior Engineer,1995-12-01,9999-01-01


In [22]:
# Query the EmployeeSQL database for Salaries
query_salary = session.query(Salaries)
salaries_df = pd.read_sql_query(query_salary.statement, session.get_bind())
salaries_df.head()

,id,emp_no,salary,from_date,to_date
0,1,10001,60117,1986-06-26,1987-06-26
1,2,10002,65828,1996-08-03,1997-08-03
2,3,10003,40006,1995-12-03,1996-12-02
3,4,10004,40054,1986-12-01,1987-12-01
4,5,10005,78228,1989-09-12,1990-09-12


In [24]:
# Join queried databases
emp_title_df = pd.merge(employee_df, titles_df, on="emp_no", how="outer")
emp_sal_title_df = pd.merge(emp_title_df, salaries_df, on="emp_no", how="outer")
emp_sal_title_df.head()


,emp_no,birth_date,first_name,last_name,gender,hire_date,id_x,title,from_date_x,to_date_x,id_y,salary,from_date_y,to_date_y
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,1,Senior Engineer,1986-06-26,9999-01-01,1,60117,1986-06-26,1987-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,2,Staff,1996-08-03,9999-01-01,2,65828,1996-08-03,1997-08-03
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,3,Senior Engineer,1995-12-03,9999-01-01,3,40006,1995-12-03,1996-12-02
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,4,Engineer,1986-12-01,1995-12-01,4,40054,1986-12-01,1987-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,5,Senior Engineer,1995-12-01,9999-01-01,4,40054,1986-12-01,1987-12-01


In [27]:
title_salary = emp_sal_title_df.groupby("title")
title_salary.count()

,emp_no,birth_date,first_name,last_name,gender,hire_date,id_x,from_date_x,to_date_x,id_y,salary,from_date_y,to_date_y
title,,,,,,,,,,,,,
Assistant Engineer,15128,15128,15128,15128,15128,15128,15128,15128,15128,15128,15128,15128,15128
Engineer,115003,115003,115003,115003,115003,115003,115003,115003,115003,115003,115003,115003,115003
Manager,24,24,24,24,24,24,24,24,24,24,24,24,24
Senior Engineer,97750,97750,97750,97750,97750,97750,97750,97750,97750,97750,97750,97750,97750
Senior Staff,92853,92853,92853,92853,92853,92853,92853,92853,92853,92853,92853,92853,92853
Staff,107391,107391,107391,107391,107391,107391,107391,107391,107391,107391,107391,107391,107391
Technique Leader,15159,15159,15159,15159,15159,15159,15159,15159,15159,15159,15159,15159,15159


In [28]:
title_salary.mean()

,emp_no,id_x,id_y,salary
title,,,,
Assistant Engineer,251495.398533,220448.507800,149076.081703,48493.204786
Engineer,252943.159987,221533.335409,149810.564750,48539.781423
Manager,110780.833333,148161.208333,100012.500000,51531.041667
Senior Engineer,253034.375949,221601.455652,149856.207090,48506.751806
Senior Staff,253423.367183,221903.988659,150061.081721,58503.286614
Staff,253399.240672,221978.378244,150111.261195,58465.271904
Technique Leader,251708.951052,220412.999010,149052.090243,48580.505772
